# Deep Learning Tutorial 1: Preprocessing

In this notebook, we will select features, sample rows and make a dataset ready for modelling.

In [78]:
from __future__ import division
import os
from glob import glob

import pandas as pd
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

### Let's have a look at the first ten rows of our data

In [35]:
pd.read_csv('data_Q3_2016/2016-07-01.csv', nrows=10)

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_22_normalized,smart_22_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2016-07-01,MJ0351YNG9Z0XA,Hitachi HDS5C3030ALA630,3000592982016,0,100,0,135.0,108.0,127,554,100,15,100,0,100,0,122.0,37.0,96,32296,100,0,NaN,NaN,100,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,319,100,319,253,23,NaN,NaN,100.0,0.0,100,0,100,0,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-07-01,MJ0351YNG9WJSA,Hitachi HDS5C3030ALA630,3000592982016,0,100,0,136.0,104.0,126,555,100,20,100,2,100,0,122.0,37.0,96,32363,100,0,NaN,NaN,100,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,358,100,358,206,29,NaN,NaN,100.0,2.0,100,0,100,0,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-07-01,PL1321LAG34XWH,Hitachi HDS5C4040ALE630,4000787030016,0,100,0,134.0,101.0,130,560,100,30,100,0,100,0,111.0,43.0,96,28956,100,0,NaN,NaN,100,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,41,100,41,200,30,NaN,NaN,100.0,0.0,100,0,100,0,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-07-01,MJ0351YNGABYAA,Hitachi HDS5C3030ALA630,3000592982016,0,100,0,136.0,104.0,137,507,100,16,100,0,100,0,122.0,37.0,96,31006,100,0,NaN,NaN,100,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,339,100,339,200,30,NaN,NaN,100.0,0.0,100,0,100,0,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-07-01,Z305B2QN,ST4000DM000,4000787030016,0,117,144377848,NaN,NaN,95,0,100,5,100,0,83,202804653,NaN,NaN,95,4778,100,0,NaN,NaN,100,5,NaN,NaN,NaN,NaN,NaN,NaN,100.0,0.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,0.0,77.0,23.0,100.0,0.0,100,0,84,33793,23,23,NaN,NaN,NaN,NaN,100,0,100,0,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,4531.0,100.0,2.230474e+10,100.0,2.562937e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2016-07-01,PL2331LAGN2YTJ,HGST HMS5C4040BLE640,4000787030016,0,100,0,133.0,104.0,100,0,100,4,100,0,100,0,111.0,43.0,98,18803,100,0,NaN,NaN,100,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,157,100,157,250,24,NaN,NaN,100.0,0.0,100,0,100,0,200,0,NaN,NaN,Na

Each drive has a serial number, a model name, and a bunch of S.M.A.R.T. attributes. As far as S.M.A.R.T is concerned, have a look at [the Wikipedia page](https://en.wikipedia.org/wiki/S.M.A.R.T.#ATA_S.M.A.R.T._attributes).

### Feature Selection

For the sake of simplicitly, the aim of this workshop, and limited RAM, we will only consider a few smart attributes which are [known to be predictive](https://www.backblaze.com/blog/what-smart-stats-indicate-hard-drive-failures/).

In [36]:
columns = [
    'date',
    'serial_number',
    'model',
    'capacity_bytes',
    'smart_1_raw',
    'smart_3_raw',
    'smart_4_raw',
    'smart_5_raw',
    'smart_7_raw',
    'smart_9_raw',
    'smart_10_raw',
    'smart_12_raw',
    'smart_187_raw',
    'smart_188_raw',
    'smart_194_raw',
    'smart_197_raw',
    'smart_198_raw',
    'smart_199_raw',
    'failure']

### Read all of our data

In [58]:
data = pd.concat((pd.read_csv(f, usecols=columns + ['model'], parse_dates=['date'])
                  for f in glob("data_Q3_2016/*.csv")), ignore_index=True)

Most hard drives don't fail

In [59]:
data.failure.value_counts()

0    6442753
1        359
Name: failure, dtype: int64

Let's keep all hard drives that have failed and a sample of the rest.

The goal of the first tutorial is to detect which hard drives have failed, not to predict which hardrives *will* fail. 

In [80]:
sample = pd.concat([data[data.failure==1], 
                    data[data.failure==0].sample(frac=0.01, random_state=42)])

In [81]:
sample.shape

(64787, 19)

### Normalization

In neural networks, it is advisable to scale attributes. Here, we do it by subtracting the mean and dividing by the standard deviation.

In [82]:
for c in columns[4:-1]:
    sample[[c]] = sample[[c]].fillna(0)
    sample[[c]] = StandardScaler().fit_transform(sample[[c]])

We encode the drive series using one-hot encoding. This helps the model learn about failures rates of drive series instead of treating each drive independently.

In [83]:
models = pd.get_dummies(sample["model"])
models.shape

(64787, 49)

In [84]:
sample.drop(["model", "serial_number"], inplace=True, axis=1)
sample.shape

(64787, 17)

In [85]:
sample = pd.concat([sample, models], axis=1)

In [86]:
sample.columns

Index([u'date', u'capacity_bytes', u'failure', u'smart_1_raw', u'smart_3_raw',
       u'smart_4_raw', u'smart_5_raw', u'smart_7_raw', u'smart_9_raw',
       u'smart_10_raw', u'smart_12_raw', u'smart_187_raw', u'smart_188_raw',
       u'smart_194_raw', u'smart_197_raw', u'smart_198_raw', u'smart_199_raw',
       u'HGST HDS5C4040ALE630', u'HGST HDS724040ALE640',
       u'HGST HMS5C4040ALE640', u'HGST HMS5C4040BLE640',
       u'HGST HUH728080ALE600', u'Hitachi HDS5C3030ALA630',
       u'Hitachi HDS5C3030BLE630', u'Hitachi HDS5C4040ALE630',
       u'Hitachi HDS722020ALA330', u'Hitachi HDS723020BLA642',
       u'Hitachi HDS723030ALA640', u'Hitachi HDS723030BLE640',
       u'Hitachi HDS724040ALE640', u'Hitachi HDT725025VLA380', u'ST2000VN000',
       u'ST250LM004 HN', u'ST31500541AS', u'ST3160316AS', u'ST3160318AS',
       u'ST320LT007', u'ST3500320AS', u'ST4000DM000', u'ST4000DX000',
       u'ST4000DX002', u'ST500LM012 HN', u'ST6000DX000', u'ST8000DM002',
       u'ST9250315AS', u'ST9320325A

In [87]:
sample.describe().T

,count,mean,std,min,25%,50%,75%,max
capacity_bytes,64787.0,4.076030e+12,1.246660e+12,1.600419e+11,4.000787e+12,4.000787e+12,4.000787e+12,8.001563e+12
failure,64787.0,5.541235e-03,7.423352e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
smart_1_raw,64787.0,-1.165523e-15,1.000008e+00,-8.665871e-01,-8.665871e-01,-4.421395e-01,8.224352e-01,3.880345e+01
smart_3_raw,64787.0,6.957097e-16,1.000008e+00,-3.143110e-01,-3.143110e-01,-3.143110e-01,1.119949e-01,8.882528e+00
smart_4_raw,64787.0,3.334694e-16,1.000008e+00,-8.623430e-02,-7.058757e-02,-4.711747e-02,1.546947e-02,2.085943e+02
smart_5_raw,64787.0,6.276046e-15,1.000008e+00,-1.596210e-02,-1.596210e-02,-1.596210e-02,-1.596210e-02,1.727084e+02
smart_7_raw,64787.0,-3.520711e-16,1.000008e+00,-9.803891e-03,-9.803891e-03,-9.656650e-03,-9.497834e-03,2.042298e+02
smart_9_raw,64787.0,-6.059127e-17,1.000008e+00,-1.317187e+00,-8.170151e-01,-1.681096e-01,5.253063e-01,3.902774e+00
smart_10_raw,64787.0,6.504332e-16,1.000008e+00,-9.644664e-03,-9.644664e-03,-9.644664e-03,-9.644664e-03,1.388329e+02
smart_12_raw,64787.0,-8.807737e-18,1.000008e+00,-1.535544e-01,-1.079977e-01,-6.244094e-02,4.385809e-02,2.491480e+02


In [88]:
sample.to_csv('sample.csv', index=False)